# Extract Content from Your File

This notebook demonstrates how to use the Content Understanding API to extract semantic content from multimodal files.

## Prerequisites
1. Ensure your Azure AI service is configured by following the [configuration steps](../README.md#configure-azure-ai-service-resource).
2. Install the required packages to run this sample.

In [ ]:
%pip install -r ../requirements.txt

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class that provides functions to interact with the Content Understanding API. Prior to the official release of the Content Understanding SDK, it serves as a lightweight SDK.
>
> Fill in the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, and **AZURE_AI_API_KEY** with the details from your Azure AI Service.

> ⚠️ Important:
You must update the code below to use your preferred Azure authentication method.
Look for the `# IMPORTANT` comments in the code and modify those sections accordingly.
Skipping this step may cause the sample to not run correctly.

> ⚠️ Note: While using a subscription key is supported, it is strongly recommended to use a token provider with Azure Active Directory (AAD) for enhanced security in production environments.

In [ ]:
import logging
import json
import os
import sys
import uuid
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based auth or subscription key; only one is required
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set it in your ".env" file if not using token authentication
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment the following line if using subscription key
    # subscription_key=AZURE_AI_API_KEY,
    x_ms_useragent="azure-ai-content-understanding-python/content_extraction",  # This header is used for sample usage telemetry; please comment out this line if you want to opt out.
)

# Utility function to save images
from PIL import Image
from io import BytesIO
import re

def save_image(image_id: str, response):
    raw_image = client.get_image_from_analyze_operation(analyze_response=response,
        image_id=image_id
    )
    image = Image.open(BytesIO(raw_image))
    # To display the image, uncomment the following line:
    # image.show()
    Path(".cache").mkdir(exist_ok=True)
    image.save(f".cache/{image_id}.jpg", "JPEG")


## Document Content

The Content Understanding API extracts all textual content from a given document file. In addition to text extraction, it performs a comprehensive layout analysis to identify and categorize tables and figures within the document. The output is presented in a structured markdown format, ensuring clarity and ease of use.

In [ ]:
ANALYZER_SAMPLE_FILE = '../data/invoice.pdf'
ANALYZER_ID = 'prebuilt-documentAnalyzer'

# Analyze document file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

> The markdown output contains detailed layout information, which is especially useful for Retrieval-Augmented Generation (RAG) scenarios. You can paste the markdown into a viewer such as Visual Studio Code to preview the layout structure.

In [ ]:
print(result_json["result"]["contents"][0]["markdown"])


> You can access layout information including `words` and `lines` within the `pages` node, paragraph details under `paragraphs`, and tables listed in the `tables` section.

In [ ]:
print(json.dumps(result_json["result"]["contents"][0], indent=2))


> This output helps you retrieve structural information about the tables embedded within the document.

In [ ]:
print(json.dumps(result_json["result"]["contents"][0]["tables"], indent=2))

## Audio Content
The API provides detailed analysis of spoken language, enabling developers to build applications such as voice recognition, customer service analytics, and conversational AI. The output structure facilitates extraction and analysis of different conversation components for further processing or insights.

Key features include:
1. **Speaker Identification:** Each phrase is linked to a specific speaker (e.g., "Speaker 2"), enabling clear differentiation in multi-participant conversations.
2. **Timing Information:** Each transcription includes precise timing data.
    - startTimeMs: The time (in milliseconds) when the phrase begins.
    - endTimeMs: The time (in milliseconds) when the phrase ends.
    This information is crucial for applications like video subtitles and audio-text synchronization.
3. **Text Content:** The actual spoken text, such as "Thank you for calling Woodgrove Travel," representing the main transcription.
4. **Confidence Score:** Each phrase has a confidence score (e.g., 0.933) indicating transcription reliability.
5. **Word-Level Breakdown:** Detailed timing for each word supports advanced speech analysis and improvements in speech recognition.
6. **Locale Specification:** The locale (e.g., "en-US") informs the transcription process of regional dialects and pronunciation nuances.

In [ ]:
ANALYZER_SAMPLE_FILE = '../data/audio.wav'
ANALYZER_ID = 'prebuilt-audioAnalyzer'

# Analyze audio file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

## Video Content
The video output provides detailed metadata about audiovisual content, specifically video shots. Key features include:

1. **Shot Information:** Each shot has a start and end time with a unique identifier. For example, 'Shot 0:0.0 to 0:2.800' includes a transcript and key frames.
2. **Transcript:** Audio transcripts formatted in WEBVTT facilitate synchronization with video playback. It captures spoken content and specifies the timing of the dialogue.
3. **Key Frames:** A collection of key frames (images) represent important moments in the video, allowing visualization of specific timestamps.
4. **Description:** Each shot includes a descriptive summary, providing context about the visuals presented. This helps in understanding the scene or subject matter without watching the video.
5. **Audio Visual Metadata:** Information such as video dimensions (width, height), type (audiovisual), and key frame timestamps.
6. **Transcript Phrases:** Specific dialog phrases with timing and speaker attribution enhance usability for applications like closed captioning and search.

In [ ]:
ANALYZER_SAMPLE_FILE = '../data/FlightSimulator.mp4'
ANALYZER_ID = 'prebuilt-videoAnalyzer'

# Analyze video file
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

# Save keyframes (optional)
keyframe_ids = set()
result_data = result_json.get("result", {})
contents = result_data.get("contents", [])

# Extract keyframe IDs from markdown content
for content in contents:
    markdown_content = content.get("markdown", "")
    if isinstance(markdown_content, str):
        keyframe_ids.update(re.findall(r"(keyFrame\.\d+)\.jpg", markdown_content))

# Output unique keyframe IDs
print("Unique Keyframe IDs:", keyframe_ids)

# Save all keyframe images
for keyframe_id in keyframe_ids:
    save_image(keyframe_id, response)

## Video Content with Face Recognition
This is a gated feature. To enable it, please follow the registration process outlined in [Azure AI Resource Face Gating](https://learn.microsoft.com/en-us/legal/cognitive-services/computer-vision/limited-access-identity?context=%2Fazure%2Fai-services%2Fcomputer-vision%2Fcontext%2Fcontext#registration-process).
In the registration form, select:
`[Video Indexer] Facial Identification (1:N or 1:1 matching)` to search for faces within media or entertainment video archives and generate metadata for these use cases.

In [ ]:
ANALYZER_SAMPLE_FILE = '../data/FlightSimulator.mp4'
ANALYZER_ID = 'prebuilt-videoAnalyzer'

# Analyze video file with face recognition
response = client.begin_analyze(ANALYZER_ID, file_location=ANALYZER_SAMPLE_FILE)
result_json = client.poll_result(response)

print(json.dumps(result_json, indent=2))

### Retrieve and Save Key Frames and Face Thumbnails

In [ ]:
# Initialize sets to store unique face IDs and keyframe IDs
face_ids = set()
keyframe_ids = set()

# Safely extract face IDs and keyframe IDs from content
result_data = result_json.get("result", {})
contents = result_data.get("contents", [])

for content in contents:
    # Extract face IDs if "faces" field exists and is a list
    faces = content.get("faces", [])
    if isinstance(faces, list):
        for face in faces:
            face_id = face.get("faceId")
            if face_id:
                face_ids.add(f"face.{face_id}")

    # Extract keyframe IDs from "markdown" if present and a string
    markdown_content = content.get("markdown", "")
    if isinstance(markdown_content, str):
        keyframe_ids.update(re.findall(r"(keyFrame\.\d+)\.jpg", markdown_content))

# Display unique face and keyframe IDs
print("Unique Face IDs:", face_ids)
print("Unique Keyframe IDs:", keyframe_ids)

# Save all face images
for face_id in face_ids:
    save_image(face_id, response)

# Save all keyframe images
for keyframe_id in keyframe_ids:
    save_image(keyframe_id, response)